In [1]:
from pyscf import ao2mo, scf, gto, fci
from openfermion import get_sparse_operator, FermionOperator, jordan_wigner
import numpy as np
import scipy as sp

In [2]:
## define single F atom!
mol_F = gto.Mole()

basis_set = 'sto-3g'
mol_F.atom = [('H', (0, 0, 0)),
              ('F', (0, 0, 1))
              ]

mol_F.basis = basis_set
mol_F.spin = 0 # <-- number of alpha e- minus number of beta e-
mol_F.build()
mol_F.verbose=1

### run HF
rhf_object = scf.RHF(mol_F)
rhf_object.kernel()

np.float64(-98.57048973900105)

In [3]:
rhf_object.mol.nao*2

np.int64(12)

In [4]:
fci_obj = fci.FCI(rhf_object)
fci_obj.kernel()
fci_obj.e_tot

np.float64(-98.6032745544027)

In [5]:
from pyscf import lo

## NOT cannonical ORBS
S_AO = rhf_object.get_ovlp()
C_mat = lo.orth.lowdin(S_AO)

In [6]:
# ## cannonical orbs
# C_mat = rhf_object.mo_coeff

In [7]:
### condition for valid ORBS:
# if False then BAD!

np.allclose(C_mat.T @ S_AO @ C_mat, np.eye(rhf_object.mol.nao))

True

In [8]:
def get_integrals(rhf_obj, C_mat):
    hcore_ao = rhf_obj.get_hcore()
    hcore_MO_spatial = C_mat.T @ hcore_ao @ C_mat

    ## manual
    ERI_AO = rhf_obj.mol.intor('int2e')
    tmp = np.einsum('pi,pqrs->iqrs', C_mat, ERI_AO, optimize=True)
    tmp = np.einsum('qa,iqrs->iars', C_mat, tmp, optimize=True)
    tmp = np.einsum('iars,rj->iajs', tmp, C_mat, optimize=True)
    eri_MO_spatial = np.einsum('iajs,sb->iajb', tmp, C_mat, optimize=True)

    ## pyscf version!
    # eri_MO_spatial2 = ao2mo.kernel(rhf_object.mol, C_mat)
    # np.allclose(eri_MO_spatial, eri_MO_spatial2)

    return hcore_MO_spatial, eri_MO_spatial


def get_spin_integrals(rhf_obj, hcore_MO_spatial, eri_MO_spatial):
    # next convert into spatial to spin
    n_spatial = rhf_obj.mol.nao # number of atomic orbitals

    n_spin = 2*n_spatial # two times for spin up and down!

    h_core_spin = np.zeros((n_spin, n_spin))
    eri_spin_chemist = np.zeros((n_spin, n_spin, n_spin, n_spin))
    for p in range(n_spatial):
        for q in range(n_spatial):
            #even indices are SPIN UP!
            h_core_spin[2*p,
                        2*q] = hcore_MO_spatial[p,q]
            
            # odd indices are SPIN DOWN!
            h_core_spin[2*p+1,
                        2*q+1] = hcore_MO_spatial[p,q]
            # note as restricted calc spin up and down are the same!
            for r in range(n_spatial):
                for s in range(n_spatial):
                    MO_spatial_term = eri_MO_spatial[p,q,r,s]
                    
                    # up, up, up, up
                    eri_spin_chemist[2*p, 2*q, 
                                    2*r, 2*s] = MO_spatial_term
                    
                    # down, down, down, down
                    eri_spin_chemist[2*p+1, 2*q+1,
                                    2*r+1, 2*s+1] = MO_spatial_term
                    
                    # mixed # <-- different here
                    eri_spin_chemist[2*p, 2*q, 
                                    2*r+1, 2*s+1] = MO_spatial_term
                    
                    # mixed <-- different here
                    eri_spin_chemist[2*p+1, 2*q+1,
                                    2*r, 2*s] = MO_spatial_term
                    
                    # other terms go to ZERO!
    return h_core_spin, eri_spin_chemist

def build_H(rhf_obj, hcore_spin, eri_spin):

    n_spin = rhf_obj.mol.nao*2
    CONSTANT_nuclear_energy = rhf_obj.energy_nuc()
    fermionic_H_chemist = FermionOperator('', CONSTANT_nuclear_energy)

    for p in range(n_spin):
        for q in range(n_spin):
            h_pq = hcore_spin[p,q]
            fermionic_H_chemist+= FermionOperator(f'{p}^ {q}', h_pq)
            
            for r in range(n_spin):
                for s in range(n_spin):
                    g_pqrs = eri_spin[p,q,r,s]
                    fermionic_H_chemist += 0.5* FermionOperator(f'{p}^ {r}^ {s} {q}', g_pqrs)
    
    return fermionic_H_chemist


In [9]:
hcore_MO_spatial, eri_MO_spatial = get_integrals(rhf_object, C_mat)
hcore_MO_spin, eri_MO_spin = get_spin_integrals(rhf_object, hcore_MO_spatial, eri_MO_spatial)
molecule_Ham1 = build_H(rhf_object, hcore_MO_spin, eri_MO_spin)

In [10]:
def build_K_mat(kappa, n_qubits):
    assert len(kappa) == (n_qubits*(n_qubits-1)//2), 'kappa vector is the wrong length'
    K = np.zeros((n_qubits,n_qubits), dtype=float)

    K[np.triu_indices(K.shape[0], k=1)] = kappa
    K += -K.conj().T

    return K

In [24]:
dim = rhf_object.mol.nao 
kappa_random = np.random.rand(dim*(dim-1)//2) # <--- random kappa vector


In [12]:
## build kappa vector manual!
kappa_random = np.zeros(dim*(dim-1)//2, dtype=np.float64)
kappa_random[0] = 1
kappa_random[1] = 2


In [25]:
K_mat = build_K_mat(kappa_random, dim)
multi_factor = 1
U = sp.linalg.expm(-K_mat*multi_factor)

In [26]:
C_mat_NEW = C_mat @ U
assert np.allclose(C_mat_NEW.T @ S_AO @ C_mat_NEW, np.eye(rhf_object.mol.nao)), 'not a valid basis!'

In [27]:
hcore_MO_spatial, eri_MO_spatial = get_integrals(rhf_object, 
                                                 C_mat_NEW)
hcore_MO_spin, eri_MO_spin = get_spin_integrals(rhf_object, hcore_MO_spatial, eri_MO_spatial)
molecule_Ham2 = build_H(rhf_object, hcore_MO_spin, eri_MO_spin)

In [28]:
len(list(molecule_Ham1)), len(list(molecule_Ham2))

(1861, 5257)

In [29]:
H_mat1 = get_sparse_operator(molecule_Ham1)
eigvals1, eigvecs1 = sp.sparse.linalg.eigsh(H_mat1, which='SA', k=1)
eigvals1

array([-98.60327455])

In [30]:
H_mat2 = get_sparse_operator(molecule_Ham2)
eigvals2, eigvecs2 = sp.sparse.linalg.eigsh(H_mat2, which='SA', k=1)
eigvals2

array([-98.60327455])

In [31]:
## solution
fci_obj.e_tot

np.float64(-98.6032745544027)

In [32]:
eigvals1 - eigvals2

array([-9.37916411e-13])

In [33]:
fci_obj.e_tot- eigvals2

array([3.41060513e-13])

In [34]:
H1q = jordan_wigner(molecule_Ham1)
H2q = jordan_wigner(molecule_Ham2)

In [35]:
H1_norm = np.abs(list(H1q.terms.values())).sum()
H2_norm = np.abs(list(H2q.terms.values())).sum()

H1_norm, H2_norm

(np.float64(137.65570531935384), np.float64(274.5738419814479))